In [1]:
import os
import io
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib
#%matplotlib inline
from matplotlib import pyplot as plt
#plt.rcParams['figure.figsize'] = 12, 10

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from PIL import Image
from IPython.display import FileLink

import datetime as dt
from datetime import date, timedelta, datetime

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.tree import DecisionTreeRegressor

from scipy import stats
import statsmodels.api as sm
from itertools import product

# import xgboost as xgb
# from xgboost import plot_tree
from matplotlib.pylab import rcParams

# from fbprophet import Prophet
# from fbprophet.make_holidays import make_holidays_df
# from fbprophet.diagnostics import cross_validation, performance_metrics
# from fbprophet.plot import plot_cross_validation_metric
# import holidays

import tsaug
from tsaug import AddNoise, Convolve, Crop, Drift, Pool, Quantize, Resize, Reverse, TimeWarp
from tsaug.visualization import plot

import warnings
warnings.simplefilter('ignore')

In [152]:
import pandas as pd
df =pd.read_csv("0_jy_interpol_v2.csv")
nwp_cols = "lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,SNOL,LSPRATE,LSSRATE,HPBLA,HFSOIL,LLRIB,UGRD,VGRD,HFSFC,XBLWS,YBLWS,LHTFL,TMP,TMIN,TMAX,SPFH,TOMFS,RH,VIS,FOGFR,DPT,PVIS5,15TL,15QT,VISIP,SHFLT-1,SHFLT-2,SHFLT-3,SHFLT-4,SHFLT-5,SHFLT-6,SHFLT-7,SHFLT-8,SHFLT-9,HTBM,SCST,DSCNC,DSCOC,NETTL-1,NETTL-2,NETTL-3,NETTL-4,NETTL-5,NETTL-6,NETTL-7,NETTL-8,NETTL-9,STOT-1,STOT-2,STOT-3,STOT-4,STOT-5,STOT-6,STOT-7,STOT-8,STOT-9,HCNP-1,HCNP-2,HCNP-3,HCNP-4,HCNP-5,WCNP-1,WCNP-2,WCNP-3,WCNP-4,WCNP-5,T15T-1,T15T-2,T15T-3,T15T-4,T15T-5,T15T-6,T15T-7,T15T-8,T15T-9,SH15-1,SH15-2,SH15-3,SH15-4,SH15-5,SH15-6,SH15-7,SH15-8,SH15-9,HDLB,STCP-1,STCP-2,STCP-3,STCP-4,STCP-5,MAXGUST,CBLT,SNOM,SMCL-1,SMCL-2,SMCL-3,SMCL-4,TSOIL-1,TSOIL-2,TSOIL-3,TSOIL-4,ROFR,SROFR,VLCDC,LCDC,MCDC,HCDC,CB25,CB45,TCAR,TCAM,CFB10,LCB,WBFLH,TCTH,50MU-min,50MV-min,50MU-max,50MV-max,PRMSL,SNOAL,TMP-SFC,HPBL,SFCR,LAND,FRICE,DIST,PRES"
nwp_cols = nwp_cols.split(',')
output_col = ["OT"]
FCST_cols =['date']
all_cols = FCST_cols+nwp_cols+output_col
# CRTN_cols = ['CRTN_TM']
# all_cols = FCST_cols+nwp_cols+output_col+CRTN_cols
tmp_df= df[all_cols]

tmp_df.date = pd.to_datetime(tmp_df.date)

# case1 
df_1= tmp_df.copy()
# df_1=df_1[df_1['lead_hr']!=0]
# df_1=df_1.drop(columns=['CRTN_TM'])
df_1.reset_index(inplace=True)
df_1=df_1.drop(columns=['index'])
df_1['month'] = df_1.date.dt.month
df_1['day']=df_1.date.dt.day
df_1['hour']=df_1.date.dt.hour
df_1

,date,lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,...,HPBL,SFCR,LAND,FRICE,DIST,PRES,OT,month,day,hour
0,2022-01-01 00:00:00,15,0.0,0.0,0.0,0.0,-97.279266,216.954483,173.384361,0.0,...,87.328502,0.500002,1,9999,393.8125,97939.48438,0.0,1,1,0
1,2022-01-01 00:00:00,27,0.0,0.0,0.0,0.0,-98.455429,219.510895,172.182388,0.0,...,87.294070,0.500002,1,9999,393.8125,97934.11719,0.0,1,1,0
2,2022-01-01 00:00:00,9,0.0,0.0,0.0,0.0,-99.215591,220.191528,173.404579,0.0,...,87.308406,0.500002,1,9999,393.8125,97986.73438,0.0,1,1,0
3,2022-01-01 00:00:00,3,0.0,0.0,0.0,0.0,-94.158768,220.376602,176.110619,0.0,...,95.338984,0.500002,1,9999,393.8125,97957.52344,0.0,1,1,0
4,2022-01-01 00:00:00,21,0.0,0.0,0.0,0.0,-98.171753,218.773865,172.580528,0.0,...,87.290813,0.500002,1,9999,393.8125,97949.35938,0.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68531,2022-12-31 23:00:00,8,0.0,0.0,0.0,0.0,-93.628098,240.974121,204.845581,0.0,...,87.287398,0.500002,1,9999,393.8125,97930.06250,0.0,12,31,23
68532,2022-12-31 23:00:00,38,0.0,0.0,0.0,0.0,-94.923172,239.007248,203.242905,0.0,...,80.867398,0.500002,1,9999,393.8125,97922.49219,0.0,12,31,23
68533,2022-12-31 23:00:00,26,0.0,0.0,0.0,0.0,-95.759827,239.917541,203.974335,0.0,...,80.827175,0.500002,1,9999,393.8125,97878.67969,0.0,12,31,23
68534,2022-12-31 23:00:00,20,0.0,0.0,0.0,0.0,-92.616913,238.594147,203.974289,0.0,...,80.866313,0.500002,1,9999,393.8125,97916.41406,0.0,12,31,23


In [148]:
check_df = df_1.groupby(['month','day','hour']).count()
check_df= check_df.reset_index()

mask_df = pd.DataFrame(check_df[check_df['date']<8][['month','day']])
aa=pd.merge(df_1,mask_df,how='inner', on=['month','day','hour'])

In [149]:
aa[['date','lead_hr']].groupby(['date'])['lead_hr']

In [150]:
bb=aa['date'].drop_duplicates().reset_index(drop=True)
bb=pd.DataFrame(bb.loc[bb.index.repeat(8)].reset_index(drop=True))
# df['text'] = df[['name','text','month']].groupby(['name','month'])['text'].transform(lambda x: ','.join(x))
# bb.to_csv('bb.csv',index=False)

In [125]:
# aa.to_csv("aa.csv",index=False)

In [151]:
aa

,date,lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,...,HPBL,SFCR,LAND,FRICE,DIST,PRES,OT,month,day,hour
0,2022-03-07 07:00:00,40,0.940918,0.025635,1.038574,1.064453,-71.015640,195.032959,214.149887,0.0,...,79.775546,0.500002,1,9999,393.8125,97222.75781,334.501944,3,7,7
1,2022-03-07 07:00:00,10,0.937988,0.025635,1.038574,1.062012,-58.788391,190.247314,215.204285,0.0,...,53.144625,0.500002,1,9999,393.8125,97250.37500,334.501944,3,7,7
2,2022-03-07 07:00:00,22,0.944824,0.011475,1.067383,1.069824,-59.240005,191.688339,214.440155,0.0,...,78.329894,0.500002,1,9999,393.8125,97294.60938,334.501944,3,7,7
3,2022-03-07 07:00:00,16,0.940918,0.014160,1.038574,1.062012,-59.319077,188.739151,216.273025,0.0,...,57.280757,0.500002,1,9999,393.8125,97279.92188,334.501944,3,7,7
4,2022-03-07 07:00:00,28,0.958008,0.002441,1.085449,1.085449,-70.522751,199.109497,209.462296,0.0,...,79.794367,0.500002,1,9999,393.8125,97302.96094,334.501944,3,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4074,2022-12-14 16:00:00,1,207.184445,184.343750,68.797493,253.145149,-110.462479,229.310791,176.950569,0.0,...,503.859105,0.500002,1,9999,393.8125,97527.53906,104.157222,12,14,16
4075,2022-12-14 16:00:00,7,201.779017,173.281250,73.483356,246.768513,-104.621262,228.580643,180.996345,0.0,...,535.210407,0.500002,1,9999,393.8125,97460.66406,104.157222,12,14,16
4076,2022-12-14 16:00:00,37,137.444959,63.156250,120.021339,183.181495,-68.697540,218.769470,219.206650,0.0,...,734.340738,0.500002,1,9999,393.8125,97421.39063,104.157222,12,14,16
4077,2022-12-14 16:00:00,25,101.129072,10.375000,123.663839,133.988058,-35.925247,210.589584,250.322090,0.0,...,860.537474,0.500002,1,9999,393.8125,97454.56250,104.157222,12,14,16


In [ ]:
def colcheck_quantile(tmp):
    tt = 0
    ori = len(tmp.columns)
    check = tmp.describe()

    for i in check.columns:
        if check[i]['25%'] == check[i]['75%']:
            tt+=1
            # print(i)
            tmp.drop(columns=i, inplace=True)

    print(f'{ori}중에 {tt}개')

    del check
    
    return tmp